In [1]:
import pandas as pd
import matplotlib as plt
import glob, os
import csv
import datetime
from geopy import distance
from config import gkey
import numpy as np
import time
import gmaps
from googlemaps import Client as GoogleMaps
import requests
gkey

'insert your api key HERE'

In [4]:
Bike_2019 = pd.read_csv("/Users/germanportes/Documents/GW_Bootcamp/HW/Project/Project_1/Q2_2019.csv")
Bike_2020 = pd.read_csv("q2_2020_df.csv")
#test = pd.read_csv("201904-capitalbikeshare-tripdata.csv")
#bike06 =pd.read_csv("202006-capitalbikeshare-tripdata 2.csv")
#test.head()
#test.info()
#print(Bike_2020.head())
Bike_2019

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,313,4/1/19 0:03,4/1/19 0:08,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W24063,Member
1,453,4/1/19 0:07,4/1/19 0:15,31119,14th & Belmont St NW,31298,15th & M St NW,W22080,Member
2,333,4/1/19 0:09,4/1/19 0:15,31203,14th & Rhode Island Ave NW,31519,1st & O St NW,W00276,Member
3,568,4/1/19 0:10,4/1/19 0:19,31202,14th & R St NW,31103,16th & Harvard St NW,W21582,Member
4,1609,4/1/19 0:10,4/1/19 0:37,31517,Union Market,31655,New Jersey Ave & F St NW,75558,Member
...,...,...,...,...,...,...,...,...,...
1035835,733,6/30/19 23:57,7/1/19 0:09,31609,Maine Ave & 7th St SW,31610,Eastern Market / 7th & North Carolina Ave SE,W22399,Casual
1035836,5127,6/30/19 23:58,7/1/19 1:23,31292,22nd St & Constitution Ave NW,31261,21st St & Constitution Ave NW,W20536,Casual
1035837,5145,6/30/19 23:58,7/1/19 1:24,31292,22nd St & Constitution Ave NW,31261,21st St & Constitution Ave NW,W20531,Casual
1035838,331,6/30/19 23:59,7/1/19 0:04,31610,Eastern Market / 7th & North Carolina Ave SE,31612,D St & Maryland Ave NE,W00795,Member


In [6]:
#Bike_2019.to_csv("/Users/germanportes/Documents/GW_Bootcamp/HW/Project/Project_1/bike_2019_df.csv")

In [5]:
#stations = test["Start station"].unique()
stations = Bike_2019["Start station"].unique()

In [6]:
for i in stations:
    Start_station = [i][0]
    print (Start_station)
    #End_station = row['End station']

4th & East Capitol St NE
14th & Belmont St NW
14th & Rhode Island Ave NW
14th & R St NW
Union Market
7th & F St NW / National Portrait Gallery
Massachusetts Ave & Dupont Circle NW
2nd St & Seaton Pl NE
M St & Pennsylvania Ave NW
Columbus Circle / Union Station
New Hampshire Ave & 24th St NW
North Capitol St & F St NW
15th & M St NW
Convention Center / 7th & M St NW
USDA / 12th & C St SW
Branch & Pennsylvania Ave SE
24th & N St NW
Maine Ave & 7th St SW
North Capitol St & G Pl NE
Rhode Island & Connecticut Ave NW
7th & R St NW / Shaw Library
21st & M St NW
20th St & Florida Ave NW
8th & H St NW
John McCormack Dr & Michigan Ave NE
14th & L St NW
Walter Reed Dr & 8th St S
Columbia Pike & S Taylor St
Vermont Ave & I St NW
11th & C St SE
15th & K St NW
12th & L St NW
15th & F St NE
Ballenger Ave & Dulaney St
Columbia & Ontario Rd NW
17th & Corcoran St NW
10th & U St NW
11th & Park Rd NW
5th & Kennedy St NW
Lincoln Park / 13th & East Capitol St NE 
D St & Maryland Ave NE
11th & Kenyon St NW
C

In [7]:
gmaps = GoogleMaps(gkey)
#test['lng'] = `""
#test['lat'] = ""
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
query_url=f'{url}query=4th & East Capitol St NE&key={gkey}'
query_url2=f'{url}{Start_station}={gkey}'
print(query_url)

ValueError: Invalid API key provided.

In [11]:
#### API loop to search for lat/lot no longer due to origina results saved as dataframe and merged into original 
#### files

# start_lat = []
# start_lng = []
# end_lat = []
# end_lng = []
# start_place = []
# end_place = []


# #for index, row in locs.iterrows():
# for i in stations:
#     try:
        
#         Start_station =[i][0]
#         #Start_station = row['Start station']
#         #End_station = row['End station']


#         response = requests.get(f'{url}query={Start_station}&key={gkey}').json()
#         #response_end = requests.get(f'{url}query={End_station}&key={gkey}').json()
#         results = response['results']
#         #results_end = response_end['results']
#         start_lat.append(results[0]['geometry']['location']['lat'])
#         start_lng.append(results[0]['geometry']['location']['lng'])
#         #locs['start_lat'] = (results[0]['geometry']['location']['lat'])
#         #locs['start_lng'] = (results[0]['geometry']['location']['lng'])
#         #locs['end_lat'] = (results_end[0]['geometry']['location']['lat'])
#         #locs['end_lng'] = (results_end[0]['geometry']['location']['lng'])
#         #end_lat.append(results_end[0]['geometry']['location']['lat'])
#         #end_lng.append(results_end[0]['geometry']['location']['lng'])
#         start_place.append(Start_station)
#         #end_place.append(End_station)
#         print (f' station name {Start_station}')
#     except:
#         print(f" this station{start_place} was not found")



 station name 4th & East Capitol St NE
 station name 14th & Belmont St NW
 station name 14th & Rhode Island Ave NW
 station name 14th & R St NW
 station name Union Market
 station name 7th & F St NW / National Portrait Gallery
 station name Massachusetts Ave & Dupont Circle NW
 station name 2nd St & Seaton Pl NE
 station name M St & Pennsylvania Ave NW
 station name Columbus Circle / Union Station
 station name New Hampshire Ave & 24th St NW
 station name North Capitol St & F St NW
 station name 15th & M St NW
 station name Convention Center / 7th & M St NW
 station name USDA / 12th & C St SW
 station name Branch & Pennsylvania Ave SE
 station name 24th & N St NW
 station name Maine Ave & 7th St SW
 station name North Capitol St & G Pl NE
 station name Rhode Island & Connecticut Ave NW
 station name 7th & R St NW / Shaw Library
 station name 21st & M St NW
 station name 20th St & Florida Ave NW
 station name 8th & H St NW
 station name John McCormack Dr & Michigan Ave NE
 station name 

 station name Offutt Ln & Chevy Chase Dr
 station name 3rd & Elm St NW
 station name Jefferson Dr & 14th St SW
 station name River Rd & Landy Ln
 station name 4th & C St SW
 station name Wisconsin Ave & O St NW
 station name 8th & D St NW
 station name Georgia Ave and Fairmont St NW
 station name 22nd & H St NW
 station name 10th & K St NW
 station name Jefferson Memorial
 station name US Dept of State / Virginia Ave & 21st St NW
 station name Henry St & Pendleton St
 station name Montello Ave & Holbrook Terr NE
 station name 3rd St & Riggs Rd NE
 station name 10th St & Constitution Ave NW
 station name Rhode Island Ave & V St NE
 station name New Jersey Ave & N St NW/Dunbar HS
 station name California St & Florida Ave NW
 station name Potomac Ave & 8th St SE
 station name 10th & E St NW
 station name 18th St & Wyoming Ave NW
 station name 15th & W St NW
 station name Maryland Ave & E St NE
 station name Fessenden St & Wisconsin Ave NW
 station name 20th & O St NW / Dupont South
 stati

 station name 1st & N St  SE
 station name Fairfax Village
 station name Fairfax Dr & Wilson Blvd
 station name Wilson Blvd & N Edgewood St
 station name Temporary Rd & Old Reston Ave
 station name 6th St & Indiana Ave NW
 station name 7th & K St NW
 station name Bethesda Metro / Wisconsin Ave & Old Georgetown Rd
 station name 19th & G St NW
 station name Woodglen Dr & Executive Blvd
 station name 18th St & Pennsylvania Ave NW
 station name Crystal Dr & Potomac Ave
 station name Bethesda Ave & Arlington Rd
 station name Market Square / King St & Royal St
 station name Fleet St & Ritchie Pkwy
 station name Virginia Ave & 25th St NW
 station name Potomac Ave & Main Line Blvd
 station name Washington-Liberty High School / N Stafford St & Generals Way
 station name 20th St & Virginia Ave NW
 station name Twinbrook Metro
 station name Potomac Ave & 35th St S
 station name Montgomery & East Ln
 station name 36th & Calvert St NW / Glover Park
 station name Arlington Blvd & Fillmore St
 statio

 station name Chillum Rd & Riggs Rd / Riggs Plaza
 station name Hyattsville Library / Adelphi Rd & Toledo Rd
 station name United Medical Center
 station name Shady Grove Hospital
 station name Joliet St & MLK Ave SW/Bald Eagle Rec Ctr
 station name Tanger Outlets
 station name Northwestern High School
 station name 61st St & Banks Pl NE
 station name Oglethorpe St & 42nd Ave
 station name Rhode Island Ave & 39th St / Brentwood Arts Exchange
 station name Largo Rd & Campus Way / Prince Georges's Comm Col
 station name Queens Chapel & Hamilton St
 station name New Hampshire Ave & East-West Hwy
 station name Riggs Rd & East West Hwy
 this station['4th & East Capitol St NE', '14th & Belmont St NW', '14th & Rhode Island Ave NW', '14th & R St NW', 'Union Market', '7th & F St NW / National Portrait Gallery', 'Massachusetts Ave & Dupont Circle NW', '2nd St & Seaton Pl NE', 'M St & Pennsylvania Ave NW', 'Columbus Circle / Union Station', 'New Hampshire Ave & 24th St NW', 'North Capitol St & F 

 station name E Fairfax St & S Washington St
 station name George Mason High School / Haycock Rd & Leesburg Pike
 station name West Falls Church Metro
 station name N Oak St & W Broad St
 station name W Broad St & Little Falls St
 station name W Columbia St & N Washington St
 station name N Roosevelt St & Roosevelt Blvd
 station name S Maple Ave & S Washington St
 station name Falls Church City Hall / Park Ave & Little Falls St
 station name Pennsylvania Ave & Park Ave
 station name Jones Branch Drive & Scotts Crossing Rd
 station name Dunn Loring Metro
 station name Merrilee Dr & Lauren Elizabeth Lane
 station name Georgia Ave & Morton St NW
 station name 18th & L St NW
 station name Rhode Island Ave & Montana Ave NE
 station name Merrifield Cinema & Merrifield Town Center
 station name nan


In [12]:
print(results)
#print(results[0]['geometry']['location']['lat'])
#print(start_lat[0])
#print (locs)

[{'business_status': 'OPERATIONAL', 'formatted_address': '20937 Ashburn Rd, Ashburn, VA 20147, United States', 'geometry': {'location': {'lat': 39.0389912, 'lng': -77.48695680000002}, 'viewport': {'northeast': {'lat': 39.04042297989272, 'lng': -77.48537057010728}, 'southwest': {'lat': 39.03772332010728, 'lng': -77.48807022989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'd7102c3aec16f6224a91d76bdd06f2875b7d64cc', 'name': 'Nan Flock', 'place_id': 'ChIJAQDAsbg-tokRYsb99RAGP68', 'plus_code': {'compound_code': '2GQ7+H6 Ashburn, Virginia', 'global_code': '87F42GQ7+H6'}, 'rating': 0, 'reference': 'ChIJAQDAsbg-tokRYsb99RAGP68', 'types': ['real_estate_agency', 'point_of_interest', 'establishment'], 'user_ratings_total': 0}, {'business_status': 'OPERATIONAL', 'formatted_address': '13021 Worldgate Dr, Herndon, VA 20170, United States', 'geometry': {'location': {'lat': 38.9570567, 'lng': -77.4005358}, 'viewport': {'northeast': {'lat': 38.958510

In [14]:
###already created out cvs in line 11/12 to avoid api search

# places = {'Start station': start_place,
#          'start_lat': start_lat,
#          'start_lng': start_lng}
# places = pd.DataFrame.from_dict(places)
# places2 = places[["Start station", 'start_lat', 'start_lng']]
# places2 = places2.rename(columns={"Start station": "End station",
#                                  "start_lat" : "end_lat",
#                                  "start_lng": "end_lng"})
# places2.to_csv("end_stations.csv")
# places.to_csv("start_stations.csv")

In [15]:
## merged original api output into dataframes lat/lon for start and end locations
places = pd.read_csv("start_stations.csv", index_col=0)
places2 = pd.read_csv("end_stations.csv", index_col=0)
Bike_2019_merged = pd.merge(Bike_2019, places, how="inner", on="Start station")
Bike_2019_merged = pd.merge(Bike_2019_merged, places2, how="inner", on= "End station")
Bike_2019_merged.sample(10)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,start_lat,start_lng,end_lat,end_lng
420805,2351,4/3/19 8:01,4/3/19 8:40,31118,3rd & Elm St NW,31636,New Jersey Ave & N St NW/Dunbar HS,W21634,Member,38.992813,-77.422990,38.903223,-77.013992
475323,475,6/20/19 13:52,6/20/19 14:00,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31204,20th & E St NW,W20445,Casual,38.890984,-77.048817,38.911006,-77.044751
582811,429,5/31/19 19:33,5/31/19 19:40,31266,11th & M St NW,31509,New Jersey Ave & R St NW,W22397,Member,38.808228,-77.263504,38.903223,-77.013992
532206,1390,6/30/19 18:17,6/30/19 18:40,31292,22nd St & Constitution Ave NW,31268,12th & U St NW,W21267,Casual,38.848909,-77.088252,28.195549,-82.648344
320022,5925,4/7/19 13:04,4/7/19 14:43,31282,16th & R St NW,31261,21st St & Constitution Ave NW,W21375,Member,39.136625,-77.709551,38.905281,-77.046684
752701,557,4/17/19 17:01,4/17/19 17:10,31260,23rd & E St NW,31062,Roosevelt Island,W21115,Member,38.906263,-77.049683,40.760503,-73.950993
623822,251,6/14/19 18:07,6/14/19 18:11,31203,14th & Rhode Island Ave NW,31214,17th & Corcoran St NW,W23436,Member,38.917555,-77.032473,38.904078,-77.037995
863668,214,6/7/19 12:28,6/7/19 12:32,31404,9th & Upshur St NW,31406,14th & Upshur St NW,W20522,Member,38.899632,-77.023812,38.917555,-77.032473
129427,931,6/8/19 12:38,6/8/19 12:53,31240,Ohio Dr & West Basin Dr SW / MLK & FDR Memorials,31314,34th & Water St NW,W20469,Member,39.996017,-83.017682,38.909869,-77.067869
978137,247,6/25/19 18:13,6/25/19 18:17,32003,Montgomery Ave & Waverly St,32055,Woodmont Ave & Strathmore St,W22447,Member,39.090360,-77.178725,38.987618,-77.095977


In [16]:
Bike_2019_merged['Start date'] = pd.to_datetime(Bike_2019_merged['Start date'])
Bike_2019_merged['End date'] = pd.to_datetime(Bike_2019_merged['End date'])
Bike_2019_merged['day_of_week'] = Bike_2019_merged['Start date'].dt.day_name()
Bike_2019_merged.sample(5)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,start_lat,start_lng,end_lat,end_lng,day_of_week
278008,1763,2019-04-15 16:29:00,2019-04-15 16:59:00,31274,10th & G St NW,31247,Jefferson Dr & 14th St SW,W21748,Casual,38.904994,-77.025760,39.037689,-77.388566,Monday
287025,250,2019-05-22 17:29:00,2019-05-22 17:33:00,31615,6th & H St NE,31611,13th & H St NE,W23678,Member,48.848337,2.328177,38.771740,-77.588143,Wednesday
1012837,1441,2019-04-15 17:07:00,2019-04-15 17:31:00,31007,Crystal City Metro / 18th St & S Bell St,31912,Mount Vernon Ave & Bruce St,W23499,Member,38.857503,-77.051819,38.826863,-77.058777,Monday
833390,376,2019-05-07 20:26:00,2019-05-07 20:32:00,31107,Lamont & Mt Pleasant NW,31104,Adams Mill & Columbia Rd NW,W23799,Member,38.931279,-77.044576,38.923786,-77.045110,Tuesday
721074,3700,2019-04-08 12:07:00,2019-04-08 13:08:00,31230,Metro Center / 12th & G St NW,31241,Thomas Circle,W00321,Member,38.898969,-77.028234,38.905366,-77.032154,Monday


In [17]:
# Extracts distance traveled from start_lat, start_lng, end_lat, end_lng
def dist_from_coordinates(lat1, lon1, lat2, lon2):
    
    R = 6371  # Earth radius in km

#conversion to radians
    d_lat = np.radians(lat2-lat1)
    d_lon = np.radians(lon2-lon1)

    r_lat1 = np.radians(lat1)
    r_lat2 = np.radians(lat2)

#haversine formula
    a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

    haversine = 2 * R * np.arcsin(np.sqrt(a))

    return haversine

new_column = []                    #empty column for distance
for index,row in Bike_2019_merged.iterrows():
    lat1 = row['start_lat'] #first row of location.lat column here
    lon1 = row['start_lng'] #first row of location.long column here
    lat2 = row['end_lat'] #second row of location.lat column here
    lon2 = row['end_lng'] #second row of location.long column here
    value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
    new_column.append(value)   #append the empty list with distance values

Bike_2019_merged.insert(14,"Distance Traveled",new_column)  #4 is the index where you want to place your column. Column index starts with 0. "Distance" is the header and new_column are the values in the column.

In [24]:
# rounds the ouput of distance traveled to 2 decimal places
Bike_2019_merged['Distance Traveled']= Bike_2019_merged['Distance Traveled'].round(2)
Bike_2019_merged

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,start_lat,start_lng,end_lat,end_lng,day_of_week,Distance Traveled,year
0,313,2019-04-01 00:03:00,2019-04-01 00:08:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W24063,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019
1,447,2019-04-01 19:35:00,2019-04-01 19:42:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22518,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019
2,423,2019-04-03 20:38:00,2019-04-03 20:45:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W01440,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019
3,298,2019-04-03 23:58:00,2019-04-04 00:03:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22313,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019
4,375,2019-04-04 19:40:00,2019-04-04 19:46:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W23620,Member,38.887837,-77.017676,48.842162,2.292766,Thursday,6161.39,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031446,1183,2019-05-29 20:38:00,2019-05-29 20:58:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W21107,Casual,39.091028,-77.204519,39.091028,-77.204519,Wednesday,0.00,2019
1031447,5358,2019-06-01 10:54:00,2019-06-01 12:24:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W22857,Member,39.091028,-77.204519,39.091028,-77.204519,Saturday,0.00,2019
1031448,4624,2019-06-16 08:37:00,2019-06-16 09:54:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W01034,Member,39.091028,-77.204519,39.091028,-77.204519,Sunday,0.00,2019
1031449,1131,2019-06-19 17:20:00,2019-06-19 17:38:00,32007,E Montgomery Ave & Maryland Ave,32005,Traville Gateway Dr & Gudelsky Dr,W21188,Member,39.084058,-77.150928,39.091028,-77.204519,Wednesday,4.69,2019


In [21]:
# extracts year from start date column
Bike_2019_merged['year'] = Bike_2019_merged['Start date'].dt.year

Bike_2019_merged.to_csv('Bike_2019_merged.csv')

In [25]:
# convert year to interger
Bike_2019_merged['year']=pd.to_numeric(Bike_2019_merged['year'])
Bike_2019_merged

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,start_lat,start_lng,end_lat,end_lng,day_of_week,Distance Traveled,year
0,313,2019-04-01 00:03:00,2019-04-01 00:08:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W24063,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019
1,447,2019-04-01 19:35:00,2019-04-01 19:42:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22518,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019
2,423,2019-04-03 20:38:00,2019-04-03 20:45:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W01440,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019
3,298,2019-04-03 23:58:00,2019-04-04 00:03:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22313,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019
4,375,2019-04-04 19:40:00,2019-04-04 19:46:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W23620,Member,38.887837,-77.017676,48.842162,2.292766,Thursday,6161.39,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031446,1183,2019-05-29 20:38:00,2019-05-29 20:58:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W21107,Casual,39.091028,-77.204519,39.091028,-77.204519,Wednesday,0.00,2019
1031447,5358,2019-06-01 10:54:00,2019-06-01 12:24:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W22857,Member,39.091028,-77.204519,39.091028,-77.204519,Saturday,0.00,2019
1031448,4624,2019-06-16 08:37:00,2019-06-16 09:54:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W01034,Member,39.091028,-77.204519,39.091028,-77.204519,Sunday,0.00,2019
1031449,1131,2019-06-19 17:20:00,2019-06-19 17:38:00,32007,E Montgomery Ave & Maryland Ave,32005,Traville Gateway Dr & Gudelsky Dr,W21188,Member,39.084058,-77.150928,39.091028,-77.204519,Wednesday,4.69,2019


In [28]:
Bike_2020.drop(["is_equity"], axis = 1, inplace =True)

In [ ]:
#test_merged.isnull().sum(axis = 0)
# test['Start date'] = pd.to_datetime(test['Start date'])
# test['End date'] = pd.to_datetime(test['End date'])
# test['day_of_week'] = test['Start date'].dt.day_name()
# print(test.sample(5))

In [26]:

Bike_2020.rename(columns={"started_at": "Start date", "ended_at": "End date", "start_station_name": "Start station","end_station_name":"End station", "member_casual": "Member type"},inplace=True)
Bike_2020


,ride_id,rideable_type,Start date,End date,Start station,start_station_id,End station,end_station_id,start_lat,start_lng,end_lat,end_lng,Member type,is_equity
0,495C10604A58ACA9,docked_bike,4/27/20 10:33,4/27/20 10:53,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.876700,-77.017800,38.899000,-77.033400,member,NaN
1,08A5138C88D182A5,docked_bike,4/27/20 11:20,4/27/20 11:42,17th & K St NW,100.0,4th & M St SW,21.0,38.902800,-77.038600,38.876700,-77.017800,member,NaN
2,505E308BDCA9C792,docked_bike,4/27/20 9:45,4/27/20 9:54,1st & M St SE,578.0,4th & M St SW,21.0,38.876900,-77.006100,38.876700,-77.017800,member,NaN
3,D3436FC6CCF04110,docked_bike,4/14/20 13:16,4/14/20 13:28,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.929500,-77.027800,38.905900,-77.032500,casual,NaN
4,37B8457C89257BD5,docked_bike,4/19/20 20:42,4/19/20 20:51,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.900400,-76.988300,38.887000,-76.996800,member,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438225,D99990073F563102,docked_bike,6/29/20 10:00,6/29/20 10:52,Gravelly Point,230.0,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.865029,-77.039511,38.890544,-77.049379,member,NaN
438226,58D09B25F2855753,docked_bike,6/15/20 11:32,6/15/20 11:42,4th & D St NW / Judiciary Square,145.0,15th & L St NW,394.0,38.895344,-77.016106,38.903660,-77.034846,member,NaN
438227,93D83692B0CDD8F4,docked_bike,6/4/20 9:46,6/4/20 10:02,Gravelly Point,230.0,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.865029,-77.039511,38.890544,-77.049379,member,NaN
438228,E6D250776A6F0276,docked_bike,6/4/20 9:20,6/4/20 9:43,Jefferson Memorial,187.0,Gravelly Point,230.0,38.879819,-77.037413,38.865029,-77.039511,member,NaN


In [29]:
Bike_2020['Start date'] = pd.to_datetime(Bike_2020['Start date'])
Bike_2020['End date'] = pd.to_datetime(Bike_2020['End date'])
Bike_2020['day_of_week'] = Bike_2020['Start date'].dt.day_name()
Bike_2020.head()


,ride_id,rideable_type,Start date,End date,Start station,start_station_id,End station,end_station_id,start_lat,start_lng,end_lat,end_lng,Member type,day_of_week
0,495C10604A58ACA9,docked_bike,2020-04-27 10:33:00,2020-04-27 10:53:00,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.8767,-77.0178,38.8990,-77.0334,member,Monday
1,08A5138C88D182A5,docked_bike,2020-04-27 11:20:00,2020-04-27 11:42:00,17th & K St NW,100.0,4th & M St SW,21.0,38.9028,-77.0386,38.8767,-77.0178,member,Monday
2,505E308BDCA9C792,docked_bike,2020-04-27 09:45:00,2020-04-27 09:54:00,1st & M St SE,578.0,4th & M St SW,21.0,38.8769,-77.0061,38.8767,-77.0178,member,Monday
3,D3436FC6CCF04110,docked_bike,2020-04-14 13:16:00,2020-04-14 13:28:00,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.9295,-77.0278,38.9059,-77.0325,casual,Tuesday
4,37B8457C89257BD5,docked_bike,2020-04-19 20:42:00,2020-04-19 20:51:00,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.9004,-76.9883,38.8870,-76.9968,member,Sunday


In [30]:
# calculates the bike ride/rental duration by subtracting end time from start time
Bike_2020['Duration']= (Bike_2020['End date'] - Bike_2020['Start date']).dt.total_seconds()
Bike_2020

,ride_id,rideable_type,Start date,End date,Start station,start_station_id,End station,end_station_id,start_lat,start_lng,end_lat,end_lng,Member type,day_of_week,Duration
0,495C10604A58ACA9,docked_bike,2020-04-27 10:33:00,2020-04-27 10:53:00,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.876700,-77.017800,38.899000,-77.033400,member,Monday,1200.0
1,08A5138C88D182A5,docked_bike,2020-04-27 11:20:00,2020-04-27 11:42:00,17th & K St NW,100.0,4th & M St SW,21.0,38.902800,-77.038600,38.876700,-77.017800,member,Monday,1320.0
2,505E308BDCA9C792,docked_bike,2020-04-27 09:45:00,2020-04-27 09:54:00,1st & M St SE,578.0,4th & M St SW,21.0,38.876900,-77.006100,38.876700,-77.017800,member,Monday,540.0
3,D3436FC6CCF04110,docked_bike,2020-04-14 13:16:00,2020-04-14 13:28:00,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.929500,-77.027800,38.905900,-77.032500,casual,Tuesday,720.0
4,37B8457C89257BD5,docked_bike,2020-04-19 20:42:00,2020-04-19 20:51:00,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.900400,-76.988300,38.887000,-76.996800,member,Sunday,540.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438225,D99990073F563102,docked_bike,2020-06-29 10:00:00,2020-06-29 10:52:00,Gravelly Point,230.0,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.865029,-77.039511,38.890544,-77.049379,member,Monday,3120.0
438226,58D09B25F2855753,docked_bike,2020-06-15 11:32:00,2020-06-15 11:42:00,4th & D St NW / Judiciary Square,145.0,15th & L St NW,394.0,38.895344,-77.016106,38.903660,-77.034846,member,Monday,600.0
438227,93D83692B0CDD8F4,docked_bike,2020-06-04 09:46:00,2020-06-04 10:02:00,Gravelly Point,230.0,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.865029,-77.039511,38.890544,-77.049379,member,Thursday,960.0
438228,E6D250776A6F0276,docked_bike,2020-06-04 09:20:00,2020-06-04 09:43:00,Jefferson Memorial,187.0,Gravelly Point,230.0,38.879819,-77.037413,38.865029,-77.039511,member,Thursday,1380.0


In [31]:
# distance traveled
def dist_from_coordinates(lat1, lon1, lat2, lon2):
    
    R = 6371  # Earth radius in km

#conversion to radians
    d_lat = np.radians(lat2-lat1)
    d_lon = np.radians(lon2-lon1)

    r_lat1 = np.radians(lat1)
    r_lat2 = np.radians(lat2)

#haversine formula
    a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

    haversine = 2 * R * np.arcsin(np.sqrt(a))

    return haversine

new_column = []                    #empty column for distance
for index,row in Bike_2020.iterrows():
    lat1 = row['start_lat'] #first row of location.lat column here
    lon1 = row['start_lng'] #first row of location.long column here
    lat2 = row['end_lat'] #second row of location.lat column here
    lon2 = row['end_lng'] #second row of location.long column here
    value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
    new_column.append(value)   #append value to new column empty list

Bike_2020.insert(14,"Distance Traveled",new_column)  
Bike_2020['Distance Traveled']= Bike_2020['Distance Traveled'].round(2)

In [32]:
Bike_2020.sample(10)

,ride_id,rideable_type,Start date,End date,Start station,start_station_id,End station,end_station_id,start_lat,start_lng,end_lat,end_lng,Member type,day_of_week,Distance Traveled
241806,CE57D2959C51B6D8,docked_bike,2020-06-28 17:45:00,2020-06-28 18:09:00,Wilson Blvd & N Edgewood St,174.0,Washington Blvd & Walter Reed Dr,360.0,38.889000,-77.092500,38.873219,-77.082104,member,Sunday,1.97
207082,939B9C0838C2FFE8,docked_bike,2020-05-12 17:14:00,2020-05-12 17:58:00,13th St & New York Ave NW,129.0,Vermont Ave & I St NW,462.0,38.900300,-77.029800,38.901100,-77.034500,member,Tuesday,0.42
210771,73A3BF86A7C78E0C,docked_bike,2020-05-28 13:16:00,2020-05-28 13:23:00,8th & H St NE,653.0,Montello Ave & Holbrook Terr NE,588.0,38.899900,-76.994900,38.907400,-76.986800,casual,Thursday,1.09
276046,460897926527D200,docked_bike,2020-06-07 20:34:00,2020-06-07 21:06:00,11th & Girard St NW,527.0,Fort Totten Metro,367.0,38.925636,-77.027112,38.952369,-77.002721,casual,Sunday,3.65
77067,5603BEF11F1F0F83,docked_bike,2020-05-30 23:27:00,2020-05-30 23:41:00,13th St & New York Ave NW,129.0,Florida Ave & R St NW,58.0,38.900300,-77.029800,38.912600,-77.013500,casual,Saturday,1.96
382894,DFF3565A3BC76244,docked_bike,2020-06-23 12:47:00,2020-06-23 13:09:00,5th & F St NW,141.0,11th & Girard St NW,527.0,38.897222,-77.019347,38.925636,-77.027112,member,Tuesday,3.23
127433,A8837216114B1323,docked_bike,2020-05-04 18:15:00,2020-05-04 18:20:00,Clarendon Blvd & N Fillmore St,176.0,Clarendon Blvd & Pierce St,170.0,38.888100,-77.093100,38.893400,-77.076400,member,Monday,1.56
402765,134201F8A0AC1E1E,docked_bike,2020-06-03 07:47:00,2020-06-03 07:55:00,7th & T St NW,22.0,15th & L St NW,394.0,38.915500,-77.022200,38.903660,-77.034846,member,Wednesday,1.71
332661,8E156D7A82DE9341,docked_bike,2020-06-16 17:12:00,2020-06-16 19:07:00,12th & Newton St NE,56.0,3rd & M St NE,191.0,38.933668,-76.991016,38.905470,-77.002130,casual,Tuesday,3.28
312908,38701F8AD73162B2,docked_bike,2020-06-06 21:44:00,2020-06-06 22:24:00,6th St & Indiana Ave NW,248.0,Georgia Ave & Morton St NW,626.0,38.894573,-77.019940,38.932128,-77.023500,casual,Saturday,4.19


In [33]:
Bike_2019_merged["Month"] = Bike_2019_merged["Start date"].dt.month
Bike_2019_merged

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,start_lat,start_lng,end_lat,end_lng,day_of_week,Distance Traveled,year,Month
0,313,2019-04-01 00:03:00,2019-04-01 00:08:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W24063,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019,4
1,447,2019-04-01 19:35:00,2019-04-01 19:42:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22518,Member,38.887837,-77.017676,48.842162,2.292766,Monday,6161.39,2019,4
2,423,2019-04-03 20:38:00,2019-04-03 20:45:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W01440,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019,4
3,298,2019-04-03 23:58:00,2019-04-04 00:03:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W22313,Member,38.887837,-77.017676,48.842162,2.292766,Wednesday,6161.39,2019,4
4,375,2019-04-04 19:40:00,2019-04-04 19:46:00,31618,4th & East Capitol St NE,31630,15th & East Capitol St NE,W23620,Member,38.887837,-77.017676,48.842162,2.292766,Thursday,6161.39,2019,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031446,1183,2019-05-29 20:38:00,2019-05-29 20:58:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W21107,Casual,39.091028,-77.204519,39.091028,-77.204519,Wednesday,0.00,2019,5
1031447,5358,2019-06-01 10:54:00,2019-06-01 12:24:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W22857,Member,39.091028,-77.204519,39.091028,-77.204519,Saturday,0.00,2019,6
1031448,4624,2019-06-16 08:37:00,2019-06-16 09:54:00,32005,Traville Gateway Dr & Gudelsky Dr,32005,Traville Gateway Dr & Gudelsky Dr,W01034,Member,39.091028,-77.204519,39.091028,-77.204519,Sunday,0.00,2019,6
1031449,1131,2019-06-19 17:20:00,2019-06-19 17:38:00,32007,E Montgomery Ave & Maryland Ave,32005,Traville Gateway Dr & Gudelsky Dr,W21188,Member,39.084058,-77.150928,39.091028,-77.204519,Wednesday,4.69,2019,6


In [ ]:
Hinas_Idea = Bike_2019_merged.groupby(['Duration', "Month"])

Germans_Idea = Bike_2019_merged["Duration"].value_counts()
Germans_Idea.plt(kind="bar", title = "German Rocks")

In [ ]:
path = "/Users/germanportes/Documents/GW_Bootcamp/Project/Group-Project_team3/Resources/Q2 2019"
path

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
bikeshare= pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
bikeshare.to_csv( "Q2_2019_df.csv", index=False, encoding='utf-8')